<a href="https://colab.research.google.com/github/jaip1157/LungCancer-Test_CT_Scan/blob/main/LungCancer_CT_Scan_Tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## !pip install fastai

In [ ]:
#importing libraries
from fastai import *
from fastai.vision import *
from fastai.vision.data import *
from fastai.vision.data import ImageDataLoaders
from fastai.metrics import error_rate
from pathlib import Path
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

In [ ]:
x  = '/content/Img/Test'
path = Path(x)
path.ls()

FileNotFoundError: [Errno 2] No such file or directory: '/content/Img/Test'

In [ ]:
from fastai.vision.all import *

np.random.seed(40)
data = ImageDataLoaders.from_folder(path, train='.', valid_pct=0.2,
                                    item_tfms=Resize(460),
                                    batch_tfms=[*aug_transforms(size=224), Normalize.from_stats(*imagenet_stats)],
                                    num_workers=4)

In [ ]:
data.show_batch(nrows=3, figsize=(7,6))

In [ ]:
data

In [ ]:
print(data.train_ds.vocab)
len(data.train_ds.vocab)
data.train_ds.c

In [ ]:
learn = cnn_learner(data, models.resnet18, metrics=[accuracy], model_dir = Path('../kaggle/working'),path = Path("."))

In [ ]:
learn.lr_find()
learn.recorder.plot_lr_find()

In [ ]:
lr1 = 1e-3
lr2 = 1e-1
learn.fit_one_cycle(30,slice(lr1,lr2))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10,slice(1e-4,1e-3))


In [ ]:
learn.recorder.plot_loss()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(6,figsize = (25,5))

In [21]:
img = PILImage.create('/content/Img/Test/Cancer/000108 (3).png')
print(learn.predict(img)[0])

Cancer


In [20]:
img = PILImage.create('/content/Img/Test/Normal/10.png')
print(learn.predict(img)[0])

Normal


In [ ]:
learn.export(file = Path("/kaggle/working/export.pkl"))
learn.model_dir = "/kaggle/working"
learn.save("stage-1",return_path=True)